In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

In [2]:
city_weather_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Manta,-0.95,-80.73,73.02,88,94,10.16,EC,1592459192
1,Bluff,-46.60,168.33,39.00,83,5,3.00,NZ,1592459093
2,Caucayá,-0.19,-74.78,71.37,97,88,0.72,CO,1592459193
3,Ushuaia,-54.80,-68.30,33.80,93,1,3.36,AR,1592458860
4,Obihiro,42.92,143.20,57.20,93,75,3.36,JP,1592459193
...,...,...,...,...,...,...,...,...,...
546,Nur-Sultan,51.18,71.45,71.60,46,75,8.95,KZ,1592459239
547,Kualakapuas,-3.01,114.39,89.60,62,40,10.29,ID,1592459239
548,Otegen Batyra,43.43,77.02,73.40,27,40,6.71,KZ,1592459239
549,Poso,-1.40,120.75,86.27,58,99,5.46,ID,1592459239


In [3]:
# Humidity Heatmap------------------------------------------------------

In [4]:
#configure gmaps
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations
locations = city_weather_df[["Lat","Lng"]]

#Add Heatmap layer to map
humidity = city_weather_df["Humidity"]

In [5]:
#Plot heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
#Create new DataFrame fitting weather criteria-----------------------------------------

In [7]:
#Narrow down the cities to fit weather conditions
#Drop any rows will null values
smaller_city_df = city_weather_df.loc[(city_weather_df["Wind Speed"] <= 10) & (city_weather_df["Clouds"]== 0) & \
                  (city_weather_df["Max Temp"] >= 70) & (city_weather_df["Max Temp"] <= 80)].dropna()
smaller_city_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
13,Nikolskoye,59.70,30.79,73.00,83,0,6.71,RU,1592459083
16,Sobolevo,54.43,31.90,70.48,82,0,8.43,RU,1592459194
19,Yulara,-25.24,130.99,71.60,15,0,9.17,AU,1592459194
39,Selma,36.57,-119.61,78.80,46,0,4.03,US,1592459043
67,Awjilah,29.11,21.29,73.47,49,0,0.92,LY,1592459199
136,Mossamedes,-15.20,12.15,71.47,51,0,4.54,AO,1592459204
268,Jalu,29.03,21.55,74.03,46,0,0.81,LY,1592459215
362,Shumskiy,54.83,99.13,72.66,44,0,9.40,RU,1592459223
367,Kiryat Gat,31.61,34.76,78.01,64,0,4.00,IL,1592459223
375,Zarechnyy,53.20,45.19,73.99,41,0,7.49,RU,1592459224


In [8]:
# Hotel Map-----------------------------------------------------

In [9]:
#Store into variable named hotel_df
hotel_df = smaller_city_df.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
13,Nikolskoye,RU,59.70,30.79,
16,Sobolevo,RU,54.43,31.90,
19,Yulara,AU,-25.24,130.99,
39,Selma,US,36.57,-119.61,
67,Awjilah,LY,29.11,21.29,
136,Mossamedes,AO,-15.20,12.15,
268,Jalu,LY,29.03,21.55,
362,Shumskiy,RU,54.83,99.13,
367,Kiryat Gat,IL,31.61,34.76,
375,Zarechnyy,RU,53.20,45.19,


In [10]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [12]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    #Due to recent events, when I am to actually google "closest hotel in...
    #I get a COVID-19 alert and a link to the location's travel advisory page
    #This naturally makes almost every result an error
    except (KeyError, IndexError):
        print("COVID-19 alert!")
        print("--------------Travel advisory for this destination")
        #delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------End of Search")

Retrieving Results for Index 13: Nikolskoye.
COVID-19 alert!
--------------Travel advisory for this destination
--------------------End of Search
Retrieving Results for Index 16: Sobolevo.
COVID-19 alert!
--------------Travel advisory for this destination
--------------------End of Search
Retrieving Results for Index 19: Yulara.
COVID-19 alert!
--------------Travel advisory for this destination
--------------------End of Search
Retrieving Results for Index 39: Selma.
COVID-19 alert!
--------------Travel advisory for this destination
--------------------End of Search
Retrieving Results for Index 67: Awjilah.
COVID-19 alert!
--------------Travel advisory for this destination
--------------------End of Search
Retrieving Results for Index 136: Mossamedes.
COVID-19 alert!
--------------Travel advisory for this destination
--------------------End of Search
Retrieving Results for Index 268: Jalu.
COVID-19 alert!
--------------Travel advisory for this destination
--------------------End of Sea

In [12]:
#Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
13,Nikolskoye,RU,59.70,30.79,
16,Sobolevo,RU,54.43,31.90,
19,Yulara,AU,-25.24,130.99,
39,Selma,US,36.57,-119.61,
67,Awjilah,LY,29.11,21.29,
136,Mossamedes,AO,-15.20,12.15,
268,Jalu,LY,29.03,21.55,
362,Shumskiy,RU,54.83,99.13,
367,Kiryat Gat,IL,31.61,34.76,
375,Zarechnyy,RU,53.20,45.19,


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Plot markers on top of the heatmap
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))